<a href="https://colab.research.google.com/github/ramfeather/googlecolab/blob/trial-2021-03-14/CNN%E3%81%8A%E8%A9%A6%E3%81%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GPUが動作しているか調べる。

In [ ]:
# pythonコードで調べる
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# linuxコマンドでマシンスペックも見れる
!cat /proc/cpuinfo

In [ ]:
# 犬猫画像を集めるライブラリをインストール
!pip install icrawler

In [ ]:
# どのライブラリが入っているか調べる
!pip list

In [ ]:
# カレントディレクトリ
!pwd

## 画像収集する

In [ ]:
from icrawler.builtin import BingImageCrawler

# 猫の画像を100枚取得。ちょっと時間かかる
crawler = BingImageCrawler(storage={"root_dir": "cat"})
crawler.crawl(keyword="猫", max_num=150)

In [ ]:
from icrawler.builtin import BingImageCrawler

# 犬の画像を100枚取得。猫もそうだが１００枚集まっていないように見える
crawler = BingImageCrawler(storage={"root_dir": "bear"})
# crawler = BingImageCrawler(storage={"root_dir": "dog"})
# crawler.crawl(keyword="犬", max_num=150)
crawler.crawl(keyword="熊", max_num=10)

In [ ]:
# サンプル表示
from IPython.display import Image,display_jpeg
# display_jpeg(Image("./cat/000002.jpg"))
display_jpeg(Image("./bear/000001.jpg"))

## データの前処理と画像分割

In [ ]:
from PIL import Image
import os, glob
import numpy as np
from PIL import ImageFile
# from IPython.display import Image as im ,display_jpeg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# IOError: image file is truncated (0 bytes not processed)回避のため
ImageFile.LOAD_TRUNCATED_IMAGES = True

classes = ["dog", "cat"]
num_classes = len(classes)
image_size = 64
# image_size = 128
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "./" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        # display_jpeg(im(file))
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        # img = mpimg.imread(‘sample.jpg’)
        # imgplot = plt.imshow(image)
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:

            # angleに代入される値
            # -20
            # -15
            # -10
            #  -5
            # 0
            # 5
            # 10
            # 15
            # 画像を5度ずつ回転
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                # FLIP_LEFT_RIGHT　は 左右反転
                img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trains)
                X_train.append(data)
                y_train.append(index)

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./dog_cat.npy", xy)

In [ ]:
# カレント直下にdog_cat.npyというデータセットができたらしい
!ls -la ./dog_cat.npy

## 学習を行う。
- 機械学習には学習と推論のフェーズがあり、まずは学習フェーズを行う。

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras.utils import np_utils
import keras
import numpy as np

classes = ["dog", "cat"]
num_classes = len(classes)
# image_size = 128
image_size = 64

"""
データを読み込む関数
"""
def load_data():
    X_train, X_test, y_train, y_test = np.load("./dog_cat.npy", allow_pickle=True)
    # 入力データの各画素値を0-1の範囲で正規化(学習コストを下げるため)
    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255
    # to_categorical()にてラベルをone hot vector化
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test

"""
モデルを学習する関数
"""
def train(X, y, X_test, y_test):
    model = Sequential()

    # Xは(1200, 64, 64, 3)
    # X.shape[1:]とすることで、(64, 64, 3)となり、入力にすることが可能です。
    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    # https://keras.io/ja/optimizers/
    # 今回は、最適化アルゴリズムにRMSpropを利用
    opt = RMSprop(lr=0.00005, decay=1e-6)
    # https://keras.io/ja/models/sequential/
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=28, epochs=40)
    # HDF5ファイルにKerasのモデルを保存
    model.save('./cnn.h5')

    return model

"""
メイン関数
データの読み込みとモデルの学習を行います。
"""
def main():
    # データの読み込み
    X_train, y_train, X_test, y_test = load_data()

    # モデルの学習
    model = train(X_train, y_train, X_test, y_test)

main()

## 推論フェーズ。犬か猫かを作ったモデルちゃんに答えてもらいます。

In [ ]:
import keras
import sys, os
import numpy as np
from keras.models import load_model
from PIL import Image

# imsize = (128, 128)
imsize = (64, 64)

"""
dog1.jpgというファイル名の画像をGoogle Colab上にアップロードする方法は2通りあります。
1つが、下記のコードを実行し画像をアップロードする方法
from google.colab import files
uploaded = files.upload()
2つが、Colab左メニューの>アイコンを押して、目次、コード スニペット、ファイル
の3つ表示されるますが、右のファイルタブから画像をアップロードする方法です。
このファイルタブをクリックするとアップロードと更新の2つがありますが、
アップロードを押すと画像をアップロードすることが可能です。
"""

# testpic     = "./dog1.jpg"
# testpic     = "./cat1.png"
testpic     = "./dog2.jpg" # やったぜこれは間違えた。猫みたいな犬の画像。猫といいよった。
keras_param = "./cnn.h5"

def load_image(path):
    img = Image.open(path)
    img = img.convert('RGB')
    # 学習時に、(64, 64, 3)で学習したので、画像の縦・横は今回 変数imsizeの(64, 64)にリサイズします。
    img = img.resize(imsize)
    # 画像データをnumpy配列の形式に変更
    img = np.asarray(img)
    img = img / 255.0
    return img

model = load_model(keras_param)
img = load_image(testpic)
prd = model.predict(np.array([img]))
print(prd) # 精度の表示
prelabel = np.argmax(prd, axis=1)
if prelabel == 0:
    print(">>> 犬")
elif prelabel == 1:
    print(">>> 猫")